# Get CPTP from Lindblad equation
# Probability Error cancellation for T1 and T2

2021 / 03 / 06 <br>
origined by rum

In [3]:
import numpy as np
import sympy
from math import pi
from random import random

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
from qiskit.visualization import plot_histogram
from qiskit.compiler import transpile, assemble

# import time
# from multiprocessing import Process
# from multiprocessing import Manager
# from multiprocessing import Pool

In [5]:
I = np.matrix([[1, 0], [0, 1]])
X = np.matrix([[0, 1], [1, 0]])
Y = np.matrix([[0, 1j], [-1j, 0]])
Z = np.matrix([[1, 0], [0, -1]])

sigma_minus = np.matrix([[0, 1], [0, 0]])
sigma_plus = sigma_minus.getH()

P = [I, X, Y, Z]
d = 2

In [9]:
np.dot(sigma_minus, np.dot(Z, sigma_plus))


matrix([[-1,  0],
        [ 0,  0]])

In [17]:
np.dot(Z, np.dot(sigma_minus,sigma_plus))


matrix([[1, 0],
        [0, 0]])

In [ ]:
## functions for Probability error cancellation

def ptm(kraus, pauli_set, d):
    """Pauli Transfar matrix
    
    Args:
        kraus: List of Kraus operator as numpy.matrix
        pauli_set: pauli operators corresponds to dimension of system
        d: dimension of system
    """
    
    if not isinstance(U, list): 
        U = [U]
    def emap(E, rho):
        e_rho = np.array([[0.0, 0.0], [0.0, 0.0]])
        for E_i in E: 
            erhoe = np.dot(E_i, np.dot(rho, E_i.getH()))
            e_rho = e_rho + erhoe
        return e_rho

    mat = []
    for P_i in pauli_set:
        row = []
        for P_j in pauli_set:
            E_ij = 1/2 * np.trace(np.dot(P_i, emap(U, P_j)))
            row.append(E_ij)
        mat.append(row)
    return np.array(mat)

# compose simultaneous equations
def solve_compose_simulequ(q_list, E_inv):
    equ_list = []
    for i in range(4): 
        for j in range(4):
            equ_ij = 0
            for l in range(16):
                equ_ij += q_list[l]*Basis_op[l][i][j]
            equ_list.append(equ_ij - E_inv[i][j])
    print("Simultaneous equations:\n")
    pprint(equ_list)
    
    solutions = sympy.solve(equ_list)
    print("\nSolutions:\n")
    print(solutions)

    return solutions

# cost of QEM (gamma)
def qem_cost(solutions):
    values = list(solutions.values())
    cost = 0
    for q_i in values:
        cost += abs(q_i)
    print("cost:", cost)
    return cost

# Quasi-probability to probability
def quasi_to_prob(q, cost):
    p = abs(q)/cost
    print(p)
    return p

def parity(q):
    if q >= 0:
        return 1
    else:
        return -1

In [ ]:
def quasi_probability_method(kraus, pauli_set, d):
    
    """
    "Error Mitigation for Short-Depth Quantum Circuits"
    https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.119.180509

    "Practical Quantum Error Mitigation for Near-Future Applications"
    https://journals.aps.org/prx/abstract/10.1103/PhysRevX.8.031027

    Args: 
        kraus: List of Kraus operator as numpy.matrix
            ex.
            # calculate Kraus oprator
            K_0 = np.sqrt(1 - p) * I
            K_1 = np.sqrt(p) * Z
            K = [K_0, K_1]
        
        
    """
    
    # calculate Pauli transfar matrix E and get inverse matrix
    E = ptm(kraus, pauli_set, d)
    E_inv = np.linalg.inv(E)

    # solve simultaneous equation to get recover probability from quasi-probability
    q_iden = sympy.Symbol('q_iden')
    q_sigmaX = sympy.Symbol('q_sigmaX')
    q_sigmaY = sympy.Symbol('q_sigmaY')
    q_sigmaZ = sympy.Symbol('q_sigmaZ')
    q_Rx = sympy.Symbol('q_Rx')
    q_Ry = sympy.Symbol('q_Ry')
    q_Rz = sympy.Symbol('q_Rz')
    q_Ryz = sympy.Symbol('q_Ryz')
    q_Rzx = sympy.Symbol('q_ Rzx')
    q_Rxy = sympy.Symbol('q_Rxy')
    q_PIx = sympy.Symbol('q_PIx')
    q_PIy = sympy.Symbol('q_PIy')
    q_PIz = sympy.Symbol('q_PIz')
    q_PIyz = sympy.Symbol('q_PIyz')
    q_PIzx = sympy.Symbol('q_PIzx')
    q_PIxy = sympy.Symbol('q_PIxy')

    q_list = [q_iden, q_sigmaX, q_sigmaY, q_sigmaZ, q_Rx, q_Ry, q_Rz, q_Ryz, q_Rzx, q_Rxy, q_PIx, q_PIy, q_PIz, q_PIyz, q_PIzx, q_PIxy]

    solutions = solve_compose_simulequ(q_list, E_inv)

    # get cost of PEC and recover probability
    cost = qem_cost(solutions)

    # P_sigmaZ = quasi_to_prob(solutions[q_sigmaZ], cost)


    
    return cost, P_sigmaZ

In [ ]:
# T1 (Amplitude damping)
def amplitude_damping():
    gamma_t = 1 - np.e^(1/T1)*gate_time

    K_0 = np.matrix([[1, 0], [0, np.sqrt(1-gamma_t)]])
    K_1 = np.matrix([[0, np.sqrt(gamma_t)], [0, 0]])
    
# T2 (Phase damping)
def insert_pd(prob, dt, circuit, qreg)
    """Insert phase damping opration"""
    # p = 1 - np.e^(-2*dt/t2)
    
    # Monte Carlo method
    r = random()
    if r<=prob:
        circuit.z(qreg)
    else:
        circuit.id(qreg)

def insert_recover_pd(P_sigmaZ, circuit, qreg, parity):
    """
    Insert recover operation for phase damping
    
    Args: 
        t2: T_2 time
        dt: delta t
        circuit: QuantumCircuit you want to insert recover operation
        qreg: QuantumRegister
        parity: 

    """
    ########## Monte Carlo method ###########
    # Choose recover operation
    r = random()
    if r<=P_sigmaZ:
        circuit.z(qreg)
        parity *= -1
    else: 
        circuit.id(qreg)
    return parity

def compose_dephasing_circuit(depth, t2, dt):

    # calculate CPTP probability from Lindblad master equation 
    p = (1 - np.e**(-2*dt*t2)) / 2
    
    ########## Quasi_probability method #########
    cost, P_sigmaZ = quasi_probability_method(kraus, pauli_set, d)

    qr = QuantumRegister(1)
    cr = ClassicalRegister(1)
    qc = QuantumCircuit(qr, cr)

    parity = 1
    cost_tot = 1
    for _ in range(depth):
        qc.x(qr[0])
        insert_pd(t2, dt, qc, qr)
        sgn = insert_recover_pd(P_sigmaZ, circuit, qreg, parity)
        qc.barrier()
        parity *= sgn
        cost_tot *= cost

    qc.measure(qr, cr)
    
    return qc, parity, cost_tot

In [ ]:
E_I = ptm(I, P, d)
E_sigmaX = ptm(X, P, d)
E_sigmaY = ptm(Y, P, d)
E_sigmaZ = ptm(Z, P, d)
E_Rx = ptm(1/np.sqrt(2) * (I + 1J*X), P, d)
E_Ry = ptm(1/np.sqrt(2) * (I + 1J*Y), P, d)
E_Rz = ptm(1/np.sqrt(2) * (I + 1J*Z), P, d)
E_Ryz = ptm(1/np.sqrt(2) * (Y + Z), P, d)
E_Rzx = ptm(1/np.sqrt(2) * (Z + X), P, d)
E_Rxy = ptm(1/np.sqrt(2) * (X + Y), P, d)
E_PIx = ptm(1/2 * (I + X), P, d)
E_PIy = ptm(1/2 * (I + Y), P, d)
E_PIz = ptm(1/2 * (I + Z), P, d)
E_PIyz = ptm(1/2 * (Y + 1j*Z), P, d)
E_PIzx = ptm(1/2 * (Z + 1j*X), P, d)
E_PIxy = ptm(1/2 * (X + 1j*Y), P, d)

# Define set of Basis operation
Basis_op = [E_I, E_sigmaX, E_sigmaY, E_sigmaZ, E_Rx, E_Ry, E_Rz, E_Ryz, E_Rzx, E_Rxy, E_PIx, E_PIy, E_PIz, E_PIyz, E_PIzx, E_PIxy]

## Simulation on Qiskit

In [ ]:
import qiskit.ignis.verification.randomized_benchmarking as rb